In [ ]:
# Steps
# 1、merge two parts				90664 sessions
#     run the merge part
# 2、remove html					76964 sessions
#     run remove_html.py
# 3、remove non-en                  41304 sessions
#     run the clean multi-lang part
# 4、split long 				    76082 segments   41308 sessions
#     run the split_long_conversations.py
# 5、remove one session             74898 segments   40994 sessions
#     run the clear len(session) == 1 part
# 6、clean completed duplicates     34435 segments   19357 sessions
#     run the clean completed duplicates part

In [ ]:
import json
import re
from tqdm import tqdm
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from collections import Counter

# Load

In [2]:
def read_json_file(file_path):
    with open(file_path, 'r', encoding="utf-8") as file:
        json_data = json.load(file)
    return json_data

# Clean

## merge two part

In [ ]:
removed_html_part1 = read_json_file('../sg_90k_part1.json')
removed_html_part2 = read_json_file('../sg_90k_part2.json')
print(len(removed_html_part1))
print(len(removed_html_part2))

merged_html_list = []
for session_dict in removed_html_part1:
    merged_html_list.append(session_dict)
for session_dict in removed_html_part1:
    merged_html_list.append(session_dict)
print(len(merged_html_list))

## clean multi-lang

In [12]:
remove_multi_lang = []
cnt_translation_task = 0
cnt_non_english = 0
for session_dict in tqdm(test, desc="Processing per sessions"):
    has_translation = any(re.search(r'\b[Tt]ranslat\w+', utter_dict['value']) for utter_dict in session_dict['conversations'] if utter_dict['from'] == 'human')
    if has_translation:
        remove_multi_lang.append(session_dict)
        cnt_translation_task += 1
    else: 
        include_non_english = any(re.search('[\u0080-\uFFFF]', utter_dict['value']) for utter_dict in session_dict['conversations'])
        if include_non_english:
            cnt_non_english += 1
        else:
            remove_multi_lang.append(session_dict)

Processing per sessions: 100%|██████████| 76964/76964 [00:04<00:00, 16780.20it/s]


In [13]:
print('Remains', len(remove_multi_lang), 'sessions.')
print('Remove', str(76964-len(remove_multi_lang)), 'sessions')
print('Remains', cnt_translation_task, 'translation tasks.')
print('Remove', cnt_non_english, 'non-english sessions.' )

Remains 41304 sessions.
Remove 35660 sessions
Remains 1584 translation tasks.
Remove 35660 non-english sessions.


## clear len(session) == 1

In [ ]:
clear_one = []
for session_dict in test:
    if len(session_dict['conversations']) > 1:
        clear_one.append(session_dict) 
print(len(clear_one))

## clean completed duplicates

In [5]:
all_rounds = []
clean_duplicates = []
for session_dict in tqdm(test, desc="Processing per sessions"):
    per_session_conv_str = ''
    for utter_dict in session_dict['conversations']:
        per_session_conv_str += utter_dict['value']
    if per_session_conv_str not in all_rounds:
        all_rounds.append(per_session_conv_str)
        clean_duplicates.append(session_dict)

Processing per sessions: 100%|██████████| 41304/41304 [00:05<00:00, 7280.75it/s] 


# Output

In [9]:
with open('final.json', 'w', encoding="utf-8" ) as file:
    json.dump(final, file, indent=2, ensure_ascii=False)